In [ ]:
import pandas as pd
import numpy as np

import utils.data as ud
import utils.plotting as up
import utils.features as uf

import tbtools.dev as tbdev

import seaborn as sns
%matplotlib inline

In [ ]:
bbh = ud.enhanced.get('bbh')

#### code for heatmap, half_hourly

In [ ]:
def heatmap(df, cmap=None, nancolor=None, **kwargs):
    cmap = sns.mpl.cm.get_cmap(cmap or 'hot')
    cmap.set_bad(nancolor or '#444455')

    sns.plt.figure(figsize=(10,10))
    return sns.heatmap(df.replace(0, np.nan), cmap=cmap, 
                linewidths=1, linecolor='#333344',
                **kwargs)
    
def hourfloat2str(hf):
    h = int(hf)
    m = int(np.round((hf % 1)*60))
    return '{:02}:{:02}'.format(h,m)
    if int(hf) < hf:
        return '{}:30'.format(int(hf))
    return '{}:00'.format(int(hf))

def half_hourly_summary_per_day(df, force_september=True, summary='count'):
    col = df.columns[0]
    df = df[col].resample('30 min')
    if summary == 'count':
        df = df.count()
    elif summary == 'mean':
        df = df.mean()
    elif summary == 'sum':
        df = df.sum()
    else:
        raise NotImplementedError()
    df = df.to_frame()
    df['Date'] = df.index.date
    df['hour'] = df.index.hour
    df['hour'] += df.index.minute / 60
    df = df.pivot(index='hour', columns='Date', values=col)
    df.index = df.index.map(hourfloat2str)
    df.index.name = 'Time of day'
    df = df.iloc[::-1]
    if force_september:
        enforce = [x.date() for x in pd.date_range('2012-09-01', '2012-10-01', freq='D')]
        for e in enforce:
            if not e in df.columns:
                df[e] = np.nan
    df = df.sort_index(axis=1)
    return df

In [ ]:
mi, ma = 0, 90

# BBH

### Line 1



#### Input line 1

In [ ]:
b = half_hourly_summary_per_day(bbh[(bbh.reinspection_change == 1) & (bbh.reinspection_change_line == 1)])
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/bbh/activity_l1_in.png')

#### Output line 1

In [ ]:
b = half_hourly_summary_per_day(bbh[(bbh.reinspection_change == -1) & (bbh.reinspection_change_line == 1)])
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/bbh/activity_l1_out.png')

### Line 2



#### Input line 2

In [ ]:
b = half_hourly_summary_per_day(bbh[(bbh.reinspection_change == 1) & (bbh.reinspection_change_line == 2)])
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/bbh/activity_l2_in.png')

#### Output line 

In [ ]:
b = half_hourly_summary_per_day(bbh[(bbh.reinspection_change == -1) & (bbh.reinspection_change_line == 2)])
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/bbh/activity_l2_out.png')

### BBH activity overall

In [ ]:
b = half_hourly_summary_per_day(bbh)
heatmap(b)
up.save_fig('data/bbh/activity_full.png')

# Almhist

In [ ]:
alm = ud.raw.get('almhist')

In [ ]:
mi, ma = None, None

In [ ]:
b = half_hourly_summary_per_day(alm[alm.AlmState==1])
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/alm/activity_raised.png')

In [ ]:
b = half_hourly_summary_per_day(alm[alm.AlmState==2])
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/alm/activity_ack.png')

In [ ]:
b = half_hourly_summary_per_day(alm)
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/alm/activity_full.png')

# OrganData
Intestine remover data

In [ ]:
od = ud.raw.get('organdata')

In [ ]:
od.groupby(['Behandlet', 'Orientering']).size().unstack()

In [ ]:
od.Laengdemaal.head()

In [ ]:
(od.Laengdemaal > od.Albuemaal).sum()

In [ ]:
od.columns

In [ ]:
b = half_hourly_summary_per_day(od)
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/od/activity.png')

# Raavaredb

In [ ]:
rvd = ud.raw.get('raavaredb')

In [ ]:
rvd.columns

In [ ]:
rvd.LeverandorNr.value_counts().head()

In [ ]:
rvd.Koen.value_counts()

In [ ]:
(rvd.Vaegt_Ukorr - rvd.Vaegt_Afregning).hist(bins=50jk)

In [ ]:
b = half_hourly_summary_per_day(rvd)
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/rvd/activity.png')

# Bemaerkningkode

In [ ]:
bmk = ud.raw.get('bemaerk')

In [ ]:
b = half_hourly_summary_per_day(bmk)
heatmap(b, vmin=mi, vmax=ma)
up.save_fig('data/bmk/activity.png')

# all together now! 

In [ ]:
import functools
b = functools.reduce(lambda x,y: x+y, 
                     map(lambda x: x.fillna(0).clip_upper(1), 
                         map(half_hourly_summary_per_day, 
                             (bbh, od, bmk, rvd, alm))))
heatmap(b, cmap=None, annot=True, cbar=False)
up.save_fig('data/activity_raw5_overlap.png')

# almhist AND bbh reinspection

In [ ]:
import functools
b = functools.reduce(lambda x,y: (x+y)==2, 
                     map(lambda x: x.fillna(0).clip_upper(1),
                         map(half_hourly_summary_per_day, 
                             (bbh[bbh.reinspection_change_line==1], alm))))

b += 2*functools.reduce(lambda x,y: (x+y)==2, 
                     map(lambda x: x.fillna(0).clip_upper(1),
                         map(half_hourly_summary_per_day, 
                             (bbh[bbh.reinspection_change_line==2], alm))))
heatmap(b, cmap=None, annot=True, cbar=False)
up.save_fig('sampling/activity_bbh_alm_overlap.png')

# Preconsolidated data

In [ ]:
pc = ud.preconsolidated.get('2012-01-01', '2012-12-31', db=1)

In [ ]:
pc2 = ud.preconsolidated.get('2012-01-01', '2012-12-31', db=2)

In [ ]:
pc = pc.set_index('Timestamp')

In [ ]:
pc2 = pc2.set_index('Timestamp')

In [ ]:
b = half_hourly_summary_per_day(pc)
ax = heatmap(b, vmin=0, vmax=450)
xtl = ax.get_xticklabels()
ax.set_xticklabels([xtl[i].get_text()[5:] if i%2==0 else '' for i in range(0,len(xtl))]);
up.save_fig('data/precons/activity_sepoct.png')

In [ ]:
b = half_hourly_summary_per_day(pc2, force_september=False)
ax = heatmap(b, vmin=0, vmax=450)
xtl = ax.get_xticklabels()
ax.set_xticklabels([xtl[i].get_text()[5:] if i%2==0 else '' for i in range(0,len(xtl))]);
up.save_fig('data/precons/activity_novdec.png')

In [ ]:
bmk.Kode.nunique()

In [ ]:
pc.head()

# Features

## $\Delta_{+} C$ and $\Delta_{-} C$, both lines

In [ ]:
import itertools
for line, d in itertools.product((1,2), (-1,1)):
    print(line, d)
    f = half_hourly_summary_per_day(
            bbh[  (bbh.reinspection_change == d) 
                & (bbh.reinspection_change_line==line)],
                              summary='count')
    heatmap(f, vmin=0, vmax=90)
    name = 'features/activity_dc_line{}_{}.png'.format(line, 'in' if d>0 else 'out')
    up.save_fig(name)


## $C$, mean per half hour

In [ ]:
for line in (1,2):
    f = half_hourly_summary_per_day(uf.C(line=line).to_frame(), summary='mean')
    heatmap(f, vmin=0, vmax=23)
    name = 'features/activity_c_line{}.png'.format(line)
    up.save_fig(name)

## $R$

In [ ]:
for line in (1,2):
    f = half_hourly_summary_per_day(uf.R(1, 10).to_frame(), summary='mean')
    heatmap(f)#, vmin=0, vmax=23)
    name = 'features/activity_r_line{}.png'.format(line)
    up.save_fig(name)

In [ ]:
df1 = ud.design_matrices.get_by_settings(sample_step='30 min', lag='0 min', dcwindow='30 min', rn=30, boawindow='30 min',
                                        line=1)

In [ ]:
df1 = pd.concat([x['x'] for x in df1.values()]).sort_index()

In [ ]:
df1.columns

In [ ]:
f = half_hourly_summary_per_day(df1[[
                                'Δ+C W=30 min L=0 min',
#                                 'R N=30 L=0 min',
#                                 'Alarm count',
                                ]],
                          summary='mean')

heatmap(f)

#### What the ...???

In [ ]:
all(f == half_hourly_summary_per_day(
        bbh[  (bbh.reinspection_change == 1) 
            & (bbh.reinspection_change_line==1)],
                          summary='count'))
# heatmap(f, annot=True)

In [ ]:
line = 2
# (half_hourly_summary_per_day(
#         bbh[  (bbh.reinspection_change == 1) 
#             & (bbh.reinspection_change_line==line)],
#                           summary='count')
# - 
half_hourly_summary_per_day(
        bbh[  (bbh.reinspection_change == -1) 
            & (bbh.reinspection_change_line==line)],
                          summary='count').max() #).sum()

## Throughput

In [ ]:
for tx, vmin, vmax in ((12,30,209),(13,30,209),(17,None,None)):
    f = half_hourly_summary_per_day(
        (bbh.Tx == tx).to_frame(), summary='sum')
    heatmap(f, vmin=vmin, vmax=vmax)
    up.save_fig('data/throughput_calendar_tx{}.png'.format(tx))

# 